In [1]:
import awkward as ak
import hist
import numpy as np
from coffea import util as cutil
from coffea.lookup_tools.dense_lookup import dense_lookup
from coffea.nanoevents import NanoAODSchema, NanoEventsFactory

btagWPs = {
    "deepJet": {
        "2016preVFP_UL": {
            "L": 0.0508,
            "M": 0.2598,
            "T": 0.6502,
        },
        "2016postVFP_UL": {
            "L": 0.0480,
            "M": 0.2489,
            "T": 0.6377,
        },
        "2017_UL": {
            "L": 0.0532,
            "M": 0.3040,
            "T": 0.7476,
        },
        "2018_UL": {
            "L": 0.0490,
            "M": 0.2783,
            "T": 0.7100,
        },
    },
    "deepCSV": {
        "2016preVFP_UL": {
            "L": 0.2027,
            "M": 0.6001,
            "T": 0.8819,
        },
        "2016postVFP_UL": {
            "L": 0.1918,
            "M": 0.5847,
            "T": 0.8767,
        },
        "2017_UL": {
            "L": 0.1355,
            "M": 0.4506,
            "T": 0.7738,
        },
        "2018_UL": {
            "L": 0.1208,
            "M": 0.4168,
            "T": 0.7665,
        },
    },
}

# single TT files to derive efficiency

taggerBranch = {"deepJet": "btagDeepFlavB", "deepCSV": "btagDeepB"}


file1 = "/uscms_data/d3/jieun201/YOURWORKINGAREA/nano_mc2016post_3-30.root"

events = NanoEventsFactory.from_root(
    file1,
entry_stop=100_000, schemaclass=NanoAODSchema,
).events()
year = '2016postVFP_UL'

#events = file1.events()

# b-tagging only applied for jets with |eta| < 2.5
phasespace_cuts = (events.Jet.pt > 30) & (abs(events.Jet.eta) < 2.5)
jets = ak.flatten(events.Jet[phasespace_cuts])

bins = [40,60,80,100,120,140,160,180,200,220,240,260,280,300,1000]


#for tag in ["deepJet", "deepCSV"]:
for tag in ["deepJet"]:
    #for wp in ["L", "M", "T"]:
    for wp in ["M"]:
        efficiencyinfo = (
            hist.Hist.new.Variable(bins, name="pt")
            .Reg(4, 0, 2.5, name="abseta")
            .IntCat([0, 4, 5], name="flavor")
            .Bool(name="passWP")
            .Double()
            .fill(
                pt=jets.pt,
                abseta=abs(jets.eta),
                flavor=jets.hadronFlavour,
                passWP=jets[taggerBranch[tag]] > btagWPs[tag][year][wp],
            )
        )
        eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
        efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
        #print(tag, wp, efflookup(np.array([42, 60]), 0.2, 2))

    #    cutil.save(efflookup, f"btageff_{tag}_{wp}_{year}.coffea")

/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1813 branches) at 0x7f6e6ac02170>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1813 branches) at 0x7f6e6ac02170>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1813 branches) at 0x7f6e6ac02170>, taking first instance
  warnings.warn(
/home/jieun201/.local/lib/python3.10/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1813 branches) at 0x7f6e6ac02170>, taking first instance
  warnings.warn(
/home/jieun201/.loca

In [2]:
print(eff)

Hist(
  Variable([40, 60, 80, 100, 120, 140, 160, 180, 200, 220, 240, 260, 280, 300, 1000], name='pt'),
  Regular(4, 0, 2.5, name='abseta'),
  IntCategory([0, 4, 5], name='flavor'),
  storage=Double()) # Sum: 53.47074014216 (nan with flow)


In [3]:
#s_2d = eff.project("pt","abseta")
#s_2d[{"flavor": 4j}]

In [4]:
eff_4=eff[{"flavor": 4j}]

In [5]:
eff_4.values()

array([[0.15840622, 0.1661055 , 0.15048544, 0.14205345],
       [0.15851955, 0.16323166, 0.14771395, 0.11378556],
       [0.14157303, 0.16877637, 0.13211382, 0.1590106 ],
       [0.15277778, 0.1425486 , 0.15789474, 0.11678832],
       [0.15649867, 0.12654321, 0.12568306, 0.12371134],
       [0.15315315, 0.14450867, 0.12631579, 0.2       ],
       [0.12582781, 0.13592233, 0.12328767, 0.15625   ],
       [0.17592593, 0.17721519, 0.        , 0.25925926],
       [0.1       , 0.10869565, 0.23809524, 0.        ],
       [0.21875   , 0.225     , 0.1875    , 0.        ],
       [0.25      , 0.05555556, 0.0625    , 0.        ],
       [0.2       , 0.16666667, 0.33333333, 0.        ],
       [0.16666667, 0.16666667, 0.33333333, 0.        ],
       [0.25      , 0.27118644, 0.42857143, 0.30769231]])

In [6]:
eff_5=eff[{"flavor": 5j}]

In [7]:
eff_5.values()

array([[0.80782238, 0.78999663, 0.73253069, 0.65414455],
       [0.82039951, 0.80343117, 0.75884544, 0.66197975],
       [0.82637307, 0.81314355, 0.75733207, 0.69823009],
       [0.83294931, 0.82127456, 0.7450565 , 0.6754386 ],
       [0.82945736, 0.83547352, 0.78453739, 0.74855491],
       [0.84069887, 0.8278481 , 0.76282051, 0.69662921],
       [0.84494774, 0.83333333, 0.77922078, 0.71681416],
       [0.82044888, 0.78169014, 0.7791411 , 0.73770492],
       [0.86585366, 0.83417085, 0.86792453, 0.76190476],
       [0.82035928, 0.86363636, 0.66197183, 0.63157895],
       [0.78991597, 0.875     , 0.81818182, 0.5625    ],
       [0.79452055, 0.81818182, 0.80769231, 0.91666667],
       [0.86885246, 0.89361702, 0.76190476, 0.72727273],
       [0.8030303 , 0.83088235, 0.71428571, 0.53333333]])

In [8]:
eff_0=eff[{"flavor": 0j}]

In [9]:
eff_0.values()

array([[0.01395099, 0.01690521, 0.01907245, 0.02544317],
       [0.00968271, 0.0097586 , 0.01477719, 0.01705191],
       [0.00672311, 0.01096738, 0.01358491, 0.01804124],
       [0.00885312, 0.01196172, 0.01154107, 0.01762115],
       [0.00723208, 0.00868878, 0.0087146 , 0.01459854],
       [0.00671892, 0.00918485, 0.00821018, 0.02108434],
       [0.01145663, 0.00564972, 0.00753769, 0.01485149],
       [0.00257732, 0.01190476, 0.01229508, 0.02238806],
       [0.01498127, 0.01321586, 0.01020408, 0.02586207],
       [0.00613497, 0.01162791, 0.        , 0.02531646],
       [0.02362205, 0.00943396, 0.02531646, 0.1       ],
       [0.01123596, 0.01428571, 0.02898551, 0.        ],
       [0.        , 0.05172414, 0.04255319, 0.03333333],
       [0.02702703, 0.04347826, 0.04020101, 0.11956522]])